https://arxiv.org/abs/1711.05772 / https://arxiv.org/abs/1802.04877

https://github.com/natashamjaques/magenta/blob/affective-reward/magenta/models/affective_reward/latent_gan.py

## Labeling data

In [1]:
#from pdb import set_trace
#set_trace()

In [2]:
import numpy as np

labels = np.empty([0, 2], dtype=int)
gens_keep = np.empty([0, ], dtype=object)
zs_keep = np.empty([0, 1, 128], dtype=float)

In [3]:
from pathlib import Path
vae_dir = Path('../../pytorch-text-vae') # https://github.com/iconix/pytorch-text-vae

n_samples = 100
temp = 0.2

%run -i $vae_dir/generate.py $vae_dir/reviews_sentences_state.pt reviews_sentences_stored_info.pkl --cache-path=$vae_dir/tmp --num-sample=$n_samples --temp=$temp

Namespace(cache_path='../../pytorch-text-vae/tmp', max_length=50, num_sample=100, print_z=False, saved_vae='../../pytorch-text-vae/reviews_sentences_state.pt', seed=None, stored_info='reviews_sentences_stored_info.pkl', temp=0.2, use_cuda=True)
Fetching cached info at ../../pytorch-text-vae/tmp/reviews_sentences_stored_info.pkl
Cache ../../pytorch-text-vae/tmp/reviews_sentences_stored_info.pkl loaded, total load time 0.6255929470062256
Found saved model ../../pytorch-text-vae/reviews_sentences_state.pt
MAX_SAMPLE: False; TRUNCATED_SAMPLE: True
Setting new random seed
---
artist s familiar with a stripped work in UNK but its its trying to put it in their listen
---
a year old producer producer i feel it below and guest vocals from vocal voice
---
the song i want to inspired by a deep house artists and i just into a real city
---
you to be released on a part of the UNK and just released their debut ep
---
UNK is a dark and as one of the love which came into a four years
---
its hard to s

---
the remix of deep house what they have taken from their remix of los angeles
---
artist has he released a long a soon as but today and the likes of trying and focus on others
---
then and in some of their debut single inspired by a little back


In [4]:
list(zip(range(len(gens)), gens))

[(0,
  'artist s familiar with a stripped work in UNK but its its trying to put it in their listen'),
 (1,
  'a year old producer producer i feel it below and guest vocals from vocal voice'),
 (2,
  'the song i want to inspired by a deep house artists and i just into a real city'),
 (3,
  'you to be released on a part of the UNK and just released their debut ep'),
 (4, 'UNK is a dark and as one of the love which came into a four years'),
 (5,
  'its hard to sucker and serves as one of the UNK is set to drop in their production'),
 (6,
  'UNK has released a release to the UNK as if youre in a pretty good alone'),
 (7,
  'i come along with a song because of the guy who has been looking for my story'),
 (8,
  'along with an fans of the track because it would be it in their creative effect'),
 (9,
  'artist has got a new remix for a year and thats exactly what i been taken from my entire album'),
 (10, 'UNK is part of the music that will blown away from the entire show'),
 (11,
  'produced

In [5]:
#new_labels = np.array([(1, -1), (10, -1)])

# -1 is "bad" because it contains a banned word, 1 is "good"
banned = ['below']
labels = np.ones(n_samples, dtype=int)
gens_lose = list(set([i for b in banned for i in np.where([b in g.split() for g in gens])[0]]))
#gens_keep = list(set(range(len(gens))) - gens_lose)
labels[gens_lose] = -1
zs_keep = zs

labels

array([ 1, -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,
       -1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [6]:
#labels = np.concatenate((labels, new_labels))
#gens_keep = np.concatenate((gens_keep, np.array(gens)[np.array([i for i, l in new_labels])]))
#zs_keep = np.concatenate((zs_keep, np.array([z.cpu().numpy() for z in zs])[np.array([i for i, l in new_labels])]))

#gens_keep

In [7]:
#gens_keep.shape, zs_keep.shape, labels.shape

In [8]:
#np.get_printoptions()

In [9]:
#np.set_printoptions(threshold=np.inf)
#zs_keep

In [10]:
# -1 is "bad", 1 is "good"
#labels = np.ones(n_samples)

# for now, make up some random labels
#np.random.seed(42)
#ones = np.random.rand(n_samples) < 0.2
#labels[ones] = -1

#labels = np.array([l for i, l in labels], dtype=float)
#labels

In [11]:
from collections import Counter
c1 = Counter([word for gen in gens for word in gen.split()])
[(b, c1[b]) for b in banned], c1.most_common(25)

([('below', 3)],
 [('the', 95),
  ('a', 76),
  ('and', 70),
  ('of', 68),
  ('to', 65),
  ('UNK', 64),
  ('artist', 26),
  ('in', 26),
  ('as', 26),
  ('with', 24),
  ('from', 23),
  ('is', 21),
  ('on', 20),
  ('it', 19),
  ('one', 16),
  ('track', 15),
  ('back', 15),
  ('into', 14),
  ('their', 13),
  ('i', 13),
  ('has', 13),
  ('up', 13),
  ('you', 12),
  ('his', 12),
  ('trying', 11)])

## Data

In [12]:
print('cuda.is_available:', torch.cuda.is_available())
print(f'available: {torch.cuda.device_count()}; current: {torch.cuda.current_device()}')
DEVICE = torch.device(f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
print('pytorch', torch.__version__)

cuda.is_available: True
available: 1; current: 0
cuda:0
pytorch 0.4.0


In [13]:
batch_size = 16

In [14]:
from fastai.dataset import *

class LatentDataset(Dataset):
    def __init__(self, x, y): self.x,self.y = x,y
    def __getitem__(self, idx): return A(self.x[idx], self.y[idx])
    def __len__(self): return len(self.x)
    
ds = LatentDataset(zs_keep, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [15]:
md.trn_ds[0]

[array([[ 0.04707,  1.08466, -0.60218, -0.67286, -0.10172,  0.0974 , -0.38054,  0.20859,  2.46602,  1.91158,
          1.5715 ,  1.08334, -0.47448,  0.50931,  1.50375, -0.68288, -0.53922, -1.55318,  1.07398,  0.16002,
          0.00061,  0.51902,  0.51798, -0.84872,  0.5032 ,  0.93224, -0.6465 ,  0.03063,  1.21098, -1.42716,
         -0.07219,  0.06316,  0.6061 ,  0.00509, -0.48093, -1.21606, -0.10246,  0.34924,  0.54495, -0.25784,
          0.98363,  0.12866,  1.65521, -0.68938,  0.21726,  0.50987,  0.75703,  0.5394 , -1.18928,  1.94751,
         -0.91054, -0.35315,  0.1198 ,  4.27383, -0.21631, -1.1195 ,  0.0644 , -0.71762,  0.06315, -1.57454,
         -1.79773, -0.95778,  0.85088,  0.4873 , -0.20203,  0.02008, -0.65778,  1.32636, -0.36775, -0.90573,
         -0.28959,  0.43247,  1.23503,  1.30368, -0.89333,  0.36211,  0.44308, -1.06952,  0.50592, -0.8578 ,
          0.27662, -0.09768, -0.17769, -0.47801,  0.35472,  1.20022, -0.48859,  0.43836,  1.32658,  0.51809,
         -0.05355, 

## Model

In [16]:
n_latent = 128
n_hidden = 1024
lr = 3e-4

In [17]:
import torch.optim as optim
import torch.nn as nn

class LCGAN_D(nn.Module):
    '''Discriminator'''
    def __init__(self, n_latent, n_hidden=n_hidden, n_output=1):
        super(LCGAN_D, self).__init__()
        
        self.i2h = nn.Linear(n_latent, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        self.h2o = nn.Linear(n_hidden, n_output)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        x = z
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        v = self.sigmoid(self.h2o(x))
        
        return v

class LCGAN_G(nn.Module):
    '''Generator'''
    def __init__(self, n_latent, n_hidden=n_hidden):
        super(LCGAN_G, self).__init__()
        self.n_latent = n_latent
        
        self.i2h = nn.Linear(n_latent, n_hidden)
        self.h2h = nn.Linear(n_hidden, n_hidden)
        # hidden-to-gating mechanism
        self.h2g = nn.Linear(n_hidden, 2*n_latent)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, z):
        x = z
        x = self.relu(self.i2h(x))
        x = self.relu(self.h2h(x))
        x = self.relu(self.h2h(x))
        x = self.h2g(x)
        
        # gating mechanism: allow network to remember/forget
        # what it wants to about the original z and x
        z_mid = x[:, self.n_latent:]
        gates = self.sigmoid(x[:, :self.n_latent])
        dz = gates * z_mid # TODO: why naming?
        z_prime = (1 - gates)*z + dz
        
        return z_prime

In [18]:
class LCGAN(nn.Module):
    def __init__(self, D, G, batch_size=batch_size):
        super(LCGAN, self).__init__()
        self.batch_size = batch_size
        
        self.D = D
        self.G = G

    def train(self):
        self.D.train()
        self.G.train()
        
    def eval(self):
        self.D.eval()
        self.G.eval()
        
    def forward(self, z=None):       
        if z is not None:
            # train discriminator
            v = self.D(z)
            return v
        else: # train GAN
            # gaussian random noise
            z_prior = torch.randn(self.batch_size, self.G.n_latent).to(DEVICE)
            
            z_prime = self.G(z_prior)
            v_prime = self.D(z_prime)
            
            return v_prime

## Training

In [19]:
gan = LCGAN(LCGAN_D(n_latent), LCGAN_G(n_latent)).to(DEVICE)

In [20]:
import fastai as fast

fast.core.set_trainable(gan.D, True)
fast.core.set_trainable(gan.G, True)

opt_d = optim.Adam(gan.D.parameters(), lr=lr)
opt_g = optim.Adam(gan.G.parameters(), lr=lr)

In [21]:
# test what the GAN is doing before any training

def generate(gan):
    gan.eval()
    with torch.no_grad():
        z = torch.randn(1, n_latent).to(DEVICE)
        z_prime = gan.G(z)

        generated = vae.decoder.generate(z_prime, 50, temp, True)
        generated_str = model.float_word_tensor_to_string(output_side, generated)

        EOS_str = f' {output_side.index_to_word(torch.LongTensor([EOS_token]))} '

        if generated_str.endswith(EOS_str):
            generated_str = generated_str[:-5]

        # flip it back
        return generated_str[::-1], z, z_prime

generate(gan)[0]

'i trying to be lost in the UNK and serves as a known for real feat'

In [22]:
for i in range(10):
    print(generate(gan)[0])

listen to UNK and serves as a UNK from the UNK are trying to look at feat
teamed up with a UNK and the dose of UNK who with some hailing from producer others
trying to speaking of the song and you might be known on the portland or alone
UNK aka artist who dropped a remix of UNK and of the song under white feat
he is listening to the kind of style and will take on in even times
in the rest aka artist recently released brand new which in in feat
artist has been trying to serves as part of the song because they look theres no feat
UNK are according to a a remix of what to fall in next
she has a listen to the rest of the track is here at times
speaking on the UNK and artist trying to with a brand new single with or feat


In [23]:
#from tqdm import tqdm

# adapted from: https://github.com/fastai/fastai/blob/master/courses/dl2/wgan.ipynb
def train(n_iter, alternate=False, first=False):
    gen_iters = 0
    for epoch in trange(n_iter):
        gan.train()
        data_iter = iter(md.trn_dl)
        i, n = 0, len(md.trn_dl)
        
        def train_G():
            ''' Train generator '''
            nonlocal gen_iters
            
            fast.core.set_trainable(gan.D, False)
            fast.core.set_trainable(gan.G, True)

            gan.G.zero_grad()

            #print(i, n)
            loss_g = - torch.log(gan()).mean()
            loss_g.backward()
            opt_g.step()
            gen_iters += 1
            
            return loss_g

        def train_D():
            ''' Train discriminator '''
            nonlocal i
            
            fast.core.set_trainable(gan.D, True)
            fast.core.set_trainable(gan.G, False)
            d_iters = 100 if (first and (gen_iters < 25) or (gen_iters % 500 == 0)) else 3
            j = 0

            while (j < d_iters) and (i < n):
                j += 1; i += 1
                batch = next(data_iter)
                #print(j, i, batch[0].size(), batch[1].size())
                z_real = batch[0].to(DEVICE)
                v = gan(z_real)

                gan.D.zero_grad()

                #loss_d = - (batch[1] * torch.log(v) + (1.0-batch[1]) * torch.log(1.0 - v)).mean()
                loss_d = - (batch[1] * torch.log(v) + (1.0-batch[1]) * torch.log(1.0 - v)).mean()
                loss_d.backward()
                opt_d.step()
                
                pbar.update()
                
            return loss_d
        
        with tqdm(total=n) as pbar:
            while i < n:
                if alternate:
                    # train discriminator
                    loss_d = train_D()
                    # then train generator a little bit
                    loss_g = train_G()
                else:
                    # train generator only
                    i += 1
                    loss_g = train_G()
                    pbar.update()
        
        if alternate:
            print(f'Loss_D {to_np(loss_d)}; Loss_G {to_np(loss_g)}; ')
        else:
            print(f'Loss_G {to_np(loss_g)}; ')

In [24]:
def train_and_generate(gan, n_epoch, alternate=False, n_sample=10):
    train(n_epoch, alternate)
    res = []
    for i in range(n_sample):
        res.append(generate(gan))
    return res

In [25]:
[res[0] for res in train_and_generate(gan, 1, alternate=True)]

100%|██████████| 7/7 [00:00<00:00, 243.08it/s]
Loss_D 0.028300680220127106; Loss_G 0.11306840181350708; 


['on the likes of UNK and it would be taken from their debut album she feat',
 'artist has a take on the side of what we have taken of their style',
 'artist has teamed up with a progressive house and i look at times',
 'artist has been listening to the UNK and one of a track that manages to live times',
 'artist is back with a remix of the ep which been looking for alone',
 'artist is taken from a brand new ep were trying to look at times',
 'the new track below and is back to the one of a look at times',
 'the track will be inspired by UNK and the UNK UNK UNK real feat',
 'the a version of UNK that we have been from their debut album or recordings',
 'reminiscent of of artist is similar from the UNK from such a look at times']

In [26]:
[res[0] for res in train_and_generate(gan, 1)]

100%|██████████| 7/7 [00:00<00:00, 405.23it/s]
Loss_G 0.02989479899406433; 


['serves with a song he song to featured in one where he recordings',
 'artist is like a listen of the house which will never really store',
 'with a a new track that seems to be taken from the UNK and jazz lyrics',
 'UNK will be serves as a deep minute or though i would be used to perfection',
 'take on UNK on serves is one of any fell in if you need to date',
 'UNK is taken from the UNK where we were talking into los city',
 'the track vocals and serves as one from the according to for more dance jam',
 'as part of his debut UNK which is according to the part of a r b style',
 'though it seems to be be sort of indie will take on any times',
 'with a full of electronic UNK and UNK is on the trying to chilled one on or ears']

In [27]:
[res[0] for res in train_and_generate(gan, 1)]

100%|██████████| 7/7 [00:00<00:00, 428.63it/s]
Loss_G 0.0002287279930897057; 


['it will be taken from my hands on this one comes to little bit',
 'this one will be on this little bit of to',
 'you will be featured on this one theres no way on this one to to bit',
 'UNK is a little different take on this one if i need to date',
 'be on this one comes in the best because i need to future',
 'as i came out as this as my didnt comes to synth',
 'this one comes in my bit on this one to comes to classic',
 'you can get my first take on this one to dance to to',
 'comes as a bit speaking on this one comes with this one comes to stay',
 'it comes in a four this comes as it comes to future synth']

In [28]:
g2 = []

for i in range(100):
    g2.append(generate(gan))
    
c2 = Counter([word for res in g2 for word in res[0].split()])
[(b, c2[b]) for b in banned], c2.most_common(25)

([('below', 0)],
 [('to', 122),
  ('this', 112),
  ('one', 91),
  ('comes', 85),
  ('on', 81),
  ('in', 40),
  ('bit', 39),
  ('you', 35),
  ('synth', 35),
  ('little', 34),
  ('can', 30),
  ('i', 28),
  ('dance', 28),
  ('be', 27),
  ('my', 23),
  ('as', 21),
  ('way', 21),
  ('it', 20),
  ('just', 19),
  ('a', 19),
  ('creative', 16),
  ('need', 13),
  ('classic', 13),
  ('featured', 12),
  ('future', 12)])

## Extras

retrain discriminator with new samples... `z_prime`s that the discriminator is still not rejecting strongly enough

In [36]:
labels = np.ones(len(g2), dtype=int)
gens_lose = list(set([i for b in banned for i in np.where([b in res[0].split() for res in g2])[0]]))
#gens_keep = list(set(range(len(g2))) - gens_lose)
labels[gens_lose] = -1
zs_keep = np.array([res[2] for res in g2], dtype=object)

labels        

array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

In [32]:
ds = LatentDataset(zs_keep, labels.astype(float))
dl = DataLoader(ds, batch_size)
md = ModelData('.', dl, None)

In [33]:
[res[0] for res in train_and_generate(gan, 1, alternate=True)]

100%|██████████| 7/7 [00:00<00:00, 284.11it/s]
Loss_D 1.193210244178772; Loss_G 1.1896806955337524; 


['artist has teams up with a a one of the UNK and it would be taken from i don no feat',
 'this with a remix of UNK and trying to take of the UNK theres no feat',
 'the remix of UNK has been trying to take of the UNK theres no feat',
 'UNK is a listen to the UNK and coming side of what to look at feat',
 'a new track called UNK and serves as one of the trying would be ready for at times',
 'artist has up with a UNK and one of the track will be inspired as no feat',
 'artist is back in the likes of UNK and serves as a no times',
 'artist who reminds back to a more stuck on the UNK and theres many feat',
 'he continues to serves as one of the week what i look no feat',
 'with a layers of UNK and and UNK of the perfect trying i look at feat']